In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import requests
import zipfile
import os

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# os.chdir('/content/drive/MyDrive/패캠강의/fastcampus_recsys_2301/Fastcampus_Codes/01_classic_recommendation/')

In [3]:
# Download and unzip the dataset
data_path = "../../../data/book_crossing/"

if not all(os.path.exists(data_path+file) for file in ['BX-Books.csv', 'BX-Users.csv', 'BX-Book-Ratings.csv']):
    url = 'http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip'
    response = requests.get(url)
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))
    zip_file.extractall(data_path)
    
else:
    print("Data Available. Just Load It!")

# Load the data
books = pd.read_csv(data_path+'BX-Books.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
users = pd.read_csv(data_path+'BX-Users.csv', sep=';', error_bad_lines=False, warn_bad_lines=False,encoding="latin-1")
ratings = pd.read_csv(data_path+'BX-Book-Ratings.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")

# Preprocess the data
ratings = ratings.merge(books[['ISBN', 'Book-Title']], on='ISBN')
ratings = ratings.drop(['ISBN'], axis=1)

Data Available. Just Load It!


/var/folders/jv/v8v_byq94r70bn81gkfc9m6m0000gn/T/ipykernel_18939/1136979317.py:14: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(data_path+'BX-Books.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
/var/folders/jv/v8v_byq94r70bn81gkfc9m6m0000gn/T/ipykernel_18939/1136979317.py:14: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(data_path+'BX-Books.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
/var/folders/jv/v8v_byq94r70bn81gkfc9m6m0000gn/T/ipykernel_18939/1136979317.py:14: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv(data_path+'BX-Books.csv', sep=';', error_bad_lines=False, warn_bad_lines=False, encoding="latin-1")
/var/fol

In [4]:
ratings.shape

(1031136, 3)

In [5]:
users['User-ID'].nunique()

278858

In [6]:
ratings.head()

,User-ID,Book-Rating,Book-Title
0,276725,0,Flesh Tones: A Novel
1,2313,5,Flesh Tones: A Novel
2,6543,0,Flesh Tones: A Novel
3,8680,5,Flesh Tones: A Novel
4,10314,9,Flesh Tones: A Novel


In [7]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [8]:
ratings['User-ID'].nunique()

92106

In [9]:
ratings['Book-Title'].nunique()

241071

In [10]:
books.ISBN.nunique()

271360

In [11]:
# Filter books with at least min_book_ratings ratings
min_book_ratings = 10
book_rating_counts = ratings['Book-Title'].value_counts()
filtered_books = book_rating_counts[book_rating_counts >= min_book_ratings].index
ratings = ratings[ratings['Book-Title'].isin(filtered_books)]

In [12]:
# Filter users who rated at least min_user_ratings books
min_user_ratings = 5
user_rating_counts = ratings['User-ID'].value_counts()
filtered_users = user_rating_counts[user_rating_counts >= min_user_ratings].index
ratings = ratings[ratings['User-ID'].isin(filtered_users)]

In [13]:
ratings.shape

(491897, 3)

In [14]:
# Create the user-item matrix
user_item_matrix = ratings.pivot_table(index='User-ID', columns='Book-Title', values='Book-Rating').fillna(0)

In [15]:
user_item_matrix.shape

(14322, 17446)

In [16]:
# Split user indices into train and test sets
user_indices = np.arange(user_item_matrix.shape[0])
train_user_indices, test_user_indices = train_test_split(user_indices, test_size=0.2, random_state=42)

In [17]:
def random_recommendation(ratings, n=10):
    unique_books = ratings['Book-Title'].unique()
    random_books = np.random.choice(unique_books, size=n, replace=False)
    return random_books

random_books = random_recommendation(ratings, n=10)
print("Random Score-Based Recommendations:")
for i, book in enumerate(random_books, 1):
    print(f"{i}. {book}")

Random Score-Based Recommendations:
1. The Glass Harmonica
2. The Debutantes
3. Death in Holy Orders : An Adam Dalgliesh Mystery
4. Celtic Magic (Llewellyn's World Magic Series)
5. The Haunting
6. The King of Torts
7. Last Exit to Brooklyn
8. The Stone Monkey : A Lincoln Rhyme Novel
9. Story of My Life
10. The Time Traveler's Wife


In [18]:
def popularity_recommendation(ratings, n=10):
    popular_books = ratings.groupby('Book-Title')['Book-Rating'].count().sort_values(ascending=False).head(n).index
    return popular_books

popular_books = popularity_recommendation(ratings, n=10)
print("\nPopularity-Based Recommendations:")
for i, book in enumerate(popular_books, 1):
    print(f"{i}. {book}")


Popularity-Based Recommendations:
1. Wild Animus
2. The Lovely Bones: A Novel
3. The Da Vinci Code
4. The Nanny Diaries: A Novel
5. Bridget Jones's Diary
6. A Painted House
7. The Secret Life of Bees
8. Divine Secrets of the Ya-Ya Sisterhood: A Novel
9. Angels &amp; Demons
10. Life of Pi


In [19]:
def recommend_books(user_index, strategy, k=10):
    if strategy == "random":
        top_k_books = random_recommendation(ratings, n=k)
    elif strategy == "popularity":
        top_k_books = popularity_recommendation(ratings, n=k)
    else:
        raise ValueError("Invalid recommendation strategy")
    return top_k_books


In [20]:
def evaluate_model(strategy, k=10):
    true_positive = 0
    false_positive = 0
    false_negative = 0

    for user_index in test_user_indices:
        true_books = set(user_item_matrix.iloc[user_index][user_item_matrix.iloc[user_index] > 0].index)
        recommended_books = set(recommend_books(user_index, strategy, k))

        tp = len(true_books.intersection(recommended_books))
        fp = len(recommended_books - true_books)
        fn = len(true_books - recommended_books)

        true_positive += tp
        false_positive += fp
        false_negative += fn

    precision = true_positive / (true_positive + false_positive)
    recall = true_positive / (true_positive + false_negative)

    return precision, recall

In [21]:
# Evaluate the random score-based recommendation model
random_precision, random_recall = evaluate_model(strategy="random")
print(f"Random Score-Based: Precision = {random_precision:.4f}, Recall = {random_recall:.4f}")

Random Score-Based: Precision = 0.0007, Recall = 0.0006


In [22]:
# Evaluate the popularity-based recommendation model
popularity_precision, popularity_recall = evaluate_model(strategy="popularity")
print(f"Popularity-Based: Precision = {popularity_precision:.4f}, Recall = {popularity_recall:.4f}")

Popularity-Based: Precision = 0.0200, Recall = 0.0167
